In [1]:
from pprint import pprint
import pandas as pd
import nltk
import regex as re

Разделение перечислений на отдельные навыки

In [65]:
not_to_split = ['a/b', 'ci/cd', 'pl/sql', 'pl/pg', 'tcp/ip', 'ui/ux', 'ux/ui', 'а/б', 'а/в', 'в/из']

In [66]:
def split_skills(skills_list):
    skills_list_new = []
    for skill in skills_list:
        skill = skill.lower()
        res = [el for el in not_to_split if(el in skill)]
        if bool(res) == False:
            if re.search('[/\,(]', skill) == None:
                skills_list_new.append(skill)
            else:
                splited_skill = re.split('[/\,(]', skill)
                for skill in splited_skill:
                    skills_list_new.append(skill)
        else:
            skills_list_new.append(skill)
    return skills_list_new


Удаление нерелевантных символов

In [5]:
def clean_text(skill):
    skill = skill.lower()
    skill = re.sub(r'1c', r'1с', skill)
    skill = re.sub(r'1[\s][сc]', r'1с', skill)
    if re.search('1с', skill) == None:
        skill = re.sub(r'[0-9]', '', skill)
    else:
        skill = re.sub('0|[2-9]', '', skill)
    res = [el for el in not_to_split if(el in skill)]
    if bool(res) == False:
        skill = re.sub(r'[^\w\s+]', ' ', skill).strip()
    skill = re.sub(r'\s+', ' ', skill)
    return skill

In [6]:
def clean_text_2(text):
    text_cleaned = []
    if type(text) == str:
        t_list = list()
        t_list.append(text)
        text = t_list
    for elem in text:
        elem = elem.lower()
        elem = re.sub(r'1c', r'1с', elem)
        elem = re.sub(r'1[\s][сc]', r'1с', elem)
        elem = re.sub('quot', '', elem)
        if re.search('1с', elem) == None:
            elem = re.sub(r'[0-9]', '', elem)
        else:
            #elem = re.sub('[0]', '', elem)
            elem = re.sub('0|[2-9]', '', elem)
        res = [el for el in not_to_split if(el in elem)]
        if bool(res) == False:
            elem = re.sub(r'[^\w\s+]', ' ', elem).strip()
        elem = re.sub(r'\s+', ' ', elem)
        text_cleaned.append(elem)
    return text_cleaned

Токенизация слов

In [67]:
from nltk import word_tokenize

Лемматизация

In [8]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [9]:
def lemmatization(word_list):
    lemm_word_list = []
    for word in word_list:
        lemm_word_list.append(morph.parse(word)[0].normal_form)
    return lemm_word_list

Стемминг

In [10]:
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language='russian')

In [11]:
def stemming(word_list):
    stemmed = []
    for word in word_list:
        stemmed.append(snowball.stem(word))
    return stemmed

Удаление стоп слов

In [74]:
from nltk.corpus import stopwords

In [13]:
sw_to_add = ['хорошее', 'знание', 'знать', 'опыт', 'работы', 'организация', 'работать', 'ms', 'условия', 'умение', 'навык']
sw_to_add = lemmatization(sw_to_add)
stopwords = stopwords.words('russian')
stopwords = stopwords + sw_to_add

In [14]:
def remove_stop_words(tokens):
    tokens = [word for word in tokens if word not in stopwords]
    return tokens

Составление n-грамм

In [15]:
from nltk.util import ngrams

In [171]:
from operator import itemgetter

def most_frequent_ngrams(ngrams_column, frequency):
    ngrams_list = ngrams_column.to_list()
    ngrams = []
    for row in ngrams_list:
        for ngram in row:
            ngrams.append(ngram)
    freqDist = nltk.FreqDist(ngrams)
    most_common = []
    for elem in freqDist.items():
        if elem[1] >= frequency:
            most_common.append(elem)
    most_common = sorted(most_common, key = itemgetter(1), reverse = True)
    return most_common     

Получение данных о скилах во всех вакансиях

In [62]:
check = pd.read_json('deduplicated.json')['key_skills']
print(len(check))
skills = []
for element in check:
    if len(element) > 0:
        skills = skills + element
print(len(skills))
#182538

17814
91711


In [39]:
skills = split_skills(skills)

In [40]:
dff = pd.DataFrame(skills)

In [41]:
dff['orig'] = dff[0]
dff = dff.drop(columns = 0)

In [42]:
dff['clean'] = dff['orig'].apply(lambda x: clean_text(x))

Не учитываем 1с, для 1с отдельно

In [43]:
dff_1c = dff.loc[dff['orig'].str.contains('1с') == True]
dff = dff.loc[dff['orig'].str.contains('1с') == False]

In [44]:
dff_1c['tokenize'] = dff_1c['clean'].apply(lambda x: word_tokenize(x, language='russian'))
dff['tokenize'] = dff['clean'].apply(lambda x: word_tokenize(x, language='russian'))

In [45]:
dff_1c['lemmatize'] = dff_1c['tokenize'].apply(lambda x: lemmatization(x))
dff['lemmatize'] = dff['tokenize'].apply(lambda x: lemmatization(x))

In [46]:
dff_1c['lemmatize'] = dff_1c['lemmatize'].apply(lambda x: remove_stop_words(x))
dff['lemmatize'] = dff['lemmatize'].apply(lambda x: remove_stop_words(x))

In [47]:
dff_1c['bigrams'] = dff_1c['lemmatize'].apply(lambda x: list(ngrams(x, 2)))
dff['bigrams'] = dff['lemmatize'].apply(lambda x: list(ngrams(x, 2)))

In [48]:
dff_1c['trigrams'] = dff_1c['lemmatize'].apply(lambda x: list(ngrams(x, 3)))
dff['trigrams'] = dff['lemmatize'].apply(lambda x: list(ngrams(x, 3)))

In [49]:
dff_1c['fourgrams'] = dff_1c['lemmatize'].apply(lambda x: list(ngrams(x, 4)))

In [172]:
most_common_1c = most_frequent_ngrams(dff_1c['lemmatize'], 50)
most_common = most_frequent_ngrams(dff['lemmatize'], 50)

In [173]:
most_common_bigrams_1c = most_frequent_ngrams(dff_1c['bigrams'], 50)
most_common_bigrams = most_frequent_ngrams(dff['bigrams'], 50)

In [183]:
most_common_trigrams_1c = most_frequent_ngrams(dff_1c['trigrams'], 50)
most_common_trigrams = most_frequent_ngrams(dff['trigrams'], 50)

In [175]:
most_common_fourgrams_1c = most_frequent_ngrams(dff_1c['fourgrams'], 50)

In [182]:
pprint(most_common_trigrams_1c)

[(('erp', 'система', 'база'), 551),
 (('система', 'база', '1с'), 551),
 (('1с', 'зарплата', 'управление'), 306),
 (('зарплата', 'управление', 'персонал'), 306),
 (('1с', 'управление', 'торговля'), 268),
 (('1с', 'управление', 'производственный'), 216),
 (('управление', 'производственный', 'предприятие'), 216),
 (('1с', 'комплексный', 'автоматизация'), 213),
 (('1с', 'управление', 'предприятие'), 188),
 (('1с', 'управление', 'холдинг'), 90),
 (('1с', 'зарплата', 'кадр'), 44),
 (('создание', 'конфигурация', '1с'), 34),
 (('1с', 'торговля', 'склад'), 31),
 (('1с', 'бухгалтерия', 'предприятие'), 23),
 (('обновление', 'конфигурация', '1с'), 23),
 (('1с', 'предприятие', 'бухгалтерский'), 23),
 (('предприятие', 'бухгалтерский', 'учёт'), 23),
 (('1с', 'управление', 'персонал'), 16),
 (('1с', 'бухгалтерия', 'склад'), 14),
 (('1с', 'crm', 'проф'), 13),
 (('1с', 'битый', 'финанс'), 13),
 (('1с', 'предприятие', 'управление'), 11),
 (('1с', 'предприятие', 'розница'), 9),
 (('предприятие', 'управлен

In [55]:
pprint(most_common)

[('анализ', 11723),
 ('sql', 5019),
 ('бизнес', 4628),
 ('аналитический', 4372),
 ('мышление', 3715),
 ('данные', 2992),
 ('информация', 2487),
 ('финансовый', 2336),
 ('процесс', 2209),
 ('разработка', 2200),
 ('системный', 2117),
 ('bpmn', 2100),
 ('powerpoint', 2025),
 ('excel', 1926),
 ('технический', 1869),
 ('больший', 1735),
 ('объём', 1675),
 ('uml', 1669),
 ('задание', 1602),
 ('аналитика', 1580),
 ('управление', 1484),
 ('исследование', 1433),
 ('atlassian', 1357),
 ('отчётность', 1333),
 ('язык', 1331),
 ('английский', 1315),
 ('моделирование', 1304),
 ('python', 1255),
 ('проект', 1165),
 ('команда', 1103),
 ('показатель', 1088),
 ('презентация', 1071),
 ('база', 1040),
 ('деловой', 1023),
 ('задача', 1016),
 ('power', 1009),
 ('разработчик', 1005),
 ('постановка', 998),
 ('jira', 962),
 ('пользователь', 939),
 ('bi', 870),
 ('подготовка', 840),
 ('многозадачность', 815),
 ('visio', 786),
 ('rest', 772),
 ('маркетинговый', 744),
 ('речь', 731),
 ('грамотный', 730),
 ('рынок

In [112]:
pprint(most_common_bigrams)

[(('аналитический', 'мышление'), 3073),
 (('бизнес', 'анализ'), 2116),
 (('анализ', 'данные'), 1742),
 (('бизнес', 'процесс'), 1722),
 (('больший', 'объём'), 1672),
 (('объём', 'информация'), 1656),
 (('технический', 'задание'), 1601),
 (('разработка', 'технический'), 1447),
 (('системный', 'анализ'), 1432),
 (('английский', 'язык'), 1301),
 (('база', 'данные'), 1013),
 (('постановка', 'задача'), 995),
 (('задача', 'разработчик'), 990),
 (('управление', 'проект'), 893),
 (('аналитический', 'исследование'), 848),
 (('моделирование', 'бизнес'), 844),
 (('atlassian', 'jira'), 787),
 (('финансовый', 'анализ'), 709),
 (('финансовый', 'показатель'), 709),
 (('power', 'bi'), 698),
 (('пользователь', 'пк'), 649),
 (('подготовка', 'презентация'), 637),
 (('грамотный', 'речь'), 619),
 (('atlassian', 'confluence'), 567),
 (('анализ', 'бизнес'), 506),
 (('сбор', 'анализ'), 505),
 (('анализ', 'информация'), 505),
 (('анализ', 'рынок'), 496),
 (('системный', 'мышление'), 474),
 (('управленческий', '

In [184]:
df_ngrams_1c = pd.concat([pd.DataFrame({'unigrams': most_common_1c}), pd.DataFrame({'bigrams': most_common_bigrams_1c}), pd.DataFrame({'trigrams': most_common_trigrams_1c}), pd.DataFrame({'fourgrams': most_common_fourgrams_1c})], axis = 1)
df_ngrams = pd.concat([pd.DataFrame({'unigrams': most_common}), pd.DataFrame({'bigrams': most_common_bigrams}), pd.DataFrame({'trigrams': most_common_trigrams})], axis = 1)

In [185]:
#df_ngrams_1c.to_excel('ngrams_1c.xlsx')
#df_ngrams.to_excel('ngrams.xlsx')

После ручной обработки n-грамм - получаем словарь

In [186]:
df_dict = pd.read_excel('ngrams-dict.xlsx', sheet_name=2, header=None)

In [189]:
df_dict[0].to_list()

['a/b тест',
 'abc анализ',
 'bb исследование',
 'bb маркетинг',
 'big data',
 'business intelligence',
 'data analysis',
 'e commerce',
 'google analytics',
 'google docs',
 'google таблица',
 'oracle pl/sql',
 'power pivot',
 'power point',
 'power query',
 'use case',
 'user story',
 'web аналитика',
 'автоматизация процесс',
 'анализ ассортиментный линейка',
 'анализ бизнес показатель',
 'анализ бизнес процесс',
 'анализ бюджет',
 'анализ данные',
 'анализ затрата',
 'анализ инвестиционный проект',
 'анализ конкурентный среда',
 'анализ поведение пользователь',
 'анализ продажа',
 'анализ риск',
 'анализ рынок',
 'анализ себестоимость',
 'анализ требование',
 'анализ финансовый показатель',
 'анализ фхд',
 'анализ ценообразование',
 'аналитика продажа',
 'аналитический исследование',
 'аналитический мышление',
 'аналитический склад ум',
 'аналитический способность',
 'английский язык',
 'аудит безопасность',
 'база данные',
 'бизнес анализ',
 'бизнес аналитика',
 'бизнес моделирова

ПЕРЕНЕСТИ КУДА_ТО

In [191]:
df_descr = pd.DataFrame({'orig': pd.read_json('deduplicated.json')['description']})

In [192]:
from nltk import sent_tokenize

In [193]:
df_descr['tokenise_sent'] = df_descr['orig'].apply(lambda x: sent_tokenize(x, language = 'russian'))

In [206]:
df_descr['clean'] = df_descr['tokenise_sent'].apply(lambda x: clean_text_2(x))

In [203]:
def tokens_from_sent_list(sent_list):
    new_sent_list = []
    for one_sent in sent_list:
        new_sent_list.append(word_tokenize(one_sent, language = 'russian'))
    return new_sent_list

In [207]:
df_descr['tokens'] = df_descr['clean'].apply(lambda x: tokens_from_sent_list(x))

In [227]:
def text_lemmatization(text):
    new_text = []
    for one in text:
        one = lemmatization(one)
        one = remove_stop_words(one)
        new_text.append(one)
    return new_text

In [229]:
df_descr['lemm_tokens'] = df_descr['tokens'].apply(lambda x: text_lemmatization(x))

KeyboardInterrupt: 

In [219]:
data_list = df_descr['tokens'].to_list()

In [220]:
for one in data_list:
    data_to_train = data_to_train + one

In [222]:
data_to_train[:50]

[['в',
  'связи',
  'с',
  'развитием',
  'сети',
  'и',
  'структуры',
  'отдела',
  'компания',
  'рассмотрит',
  'специалистов',
  'в',
  'отдел',
  'аналитики'],
 ['у',
  'нас',
  'много',
  'проектов',
  'и',
  'задач',
  'по',
  'автоматизации',
  'и',
  'оптимизации',
  'процессов',
  'в',
  'структуре',
  'и',
  'цепочке',
  'закупа',
  'управления',
  'товарными',
  'запасами'],
 ['мы',
  'открыты',
  'к',
  'обсуждению',
  'задач',
  'и',
  'условий',
  'для',
  'высоко',
  'компетентных',
  'специалистов'],
 ['уровень',
  'заработной',
  'платы',
  'обсуждается',
  'по',
  'результатам',
  'оценки',
  'компетенций'],
 ['просим', 'указывать', 'в', 'резюме', 'ваши', 'финансовые', 'ожидания'],
 ['обязанности',
  'сбор',
  'обработка',
  'и',
  'анализ',
  'больших',
  'объемов',
  'данных',
  'в',
  'excel',
  'анализ',
  'трендов',
  'прогнозирование',
  'участие',
  'в',
  'автоматизации',
  'процессов',
  'тестирование',
  'системы',
  'анализ',
  'причин',
  'отклонений',
 